## Stock Moving Average & Performance

### Case 1 - Data Exploration

In [ ]:
def case1(financial_data):
    print(financial_data.head())
    print(financial_data.tail())
    print(financial_data.describe())
    # Print First 5 rows of MSFT
    # Print Last 5 rows of MSFT
    # Print Describe MSFT

In [ ]:
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import datetime
import calendar

In [ ]:
start_date =  '2001-01-01'
end_date = '2017-12-30'

MSFT = pdr.get_data_yahoo("MSFT", start = start_date, end = end_date)

In [ ]:
MSFT.head()

In [ ]:
MSFT.index.name

### Case 2 - Monthly Average Data

In [ ]:
def case2(MSFT):
    def last_day_of_month(any_day):
        next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
        return next_month - datetime.timedelta(days=next_month.day)
    
    result = []
    for year in range(2001,2018):
        for month in range(1, 13):
            result.append(str(last_day_of_month(datetime.date(year, month, 1))))
            
    MSFT['year'] = pd.DatetimeIndex(MSFT.index).year
    MSFT['month'] = pd.DatetimeIndex(MSFT.index).month
    a = MSFT.groupby(['year','month']).mean()
    a.index = result
    a.index.name = 'Date'
    print(a.head())

In [ ]:
def case2(MSFT):
    def Date_index():
        year = list(range(2001,2018))
        month = list(range(1,13))
        result = []
        for y in year:
            for m in month:
                day = calendar.monthlen(y,m)
                date = str(datetime.date(y,m,day))
                result.append(date)
        return(result)
            
    MSFT['year'] = pd.DatetimeIndex(MSFT.index).year
    MSFT['month'] = pd.DatetimeIndex(MSFT.index).month
    a = MSFT.groupby(['year','month']).mean()
    a.index = Date_index()
    a.index.name = 'Date'
    print(a.head())

In [ ]:
case2(MSFT)

### Case 3 - Daily Close Increase%

In [ ]:
def case3(MSFT):
    MSFT['daily_close'] = (MSFT['Adj Close']-MSFT['Adj Close'].shift(1))/MSFT['Adj Close'].shift(1)
    MSFT['Adj Close'] = MSFT['daily_close']
    print(MSFT[['Adj Close']].iloc[1:,] )

In [ ]:
case3(MSFT)

### Case 4 - Cummulative Daily Return

In [ ]:
start_date =  '2001-01-01'
end_date = '2017-12-30'

MSFT = pdr.get_data_yahoo("MSFT", start = start_date, end = end_date)

In [ ]:
def case4(MSFT):
    MSFT['daily_close'] = (MSFT['Adj Close']-MSFT['Adj Close'].shift(1))/MSFT['Adj Close'].shift(1)
    MSFT['Commu_Return_D'] = (MSFT['daily_close']+1).cumprod()
    MSFT['Adj Close'] = MSFT['Commu_Return_D']
    print(MSFT[['Adj Close']].iloc[1:,] )

In [ ]:
case4(MSFT)

In [ ]:
MSFT['year'] = pd.DatetimeIndex(MSFT.index).year
MSFT['month'] = pd.DatetimeIndex(MSFT.index).month

In [ ]:
MSFT['daily_close'] = (MSFT['Adj Close']-MSFT['Adj Close'].shift(1))/MSFT['Adj Close'].shift(1)

In [ ]:
MSFT.head()

In [ ]:
MSFT['Commu_Return_D'] = (MSFT['daily_close']+1).cumprod()

In [ ]:
MSFT.head()

### Case 5 - Commulative Monthly Return

In [ ]:
def case5(MSFT):
    MSFT['year'] = pd.DatetimeIndex(MSFT.index).year
    MSFT['month'] = pd.DatetimeIndex(MSFT.index).month
    MSFT['daily_close'] = (MSFT['Adj Close']-MSFT['Adj Close'].shift(1))/MSFT['Adj Close'].shift(1)
    MSFT['Commu_Return_D'] = (MSFT['daily_close']+1).cumprod()
    MSFT['Adj Close'] = MSFT['Commu_Return_D']
    Commu_Return_M = MSFT.groupby(['year','month'])[['Adj Close']].mean()
    Commu_Return_M.index = Date_index()
    Commu_Return_M.index.name = 'Date'
    print(Commu_Return_M)

In [ ]:
MSFT.groupby(['year','month'])[['Commu_Return_D']].mean()

### Case 6 - Moving Average of 20

In [ ]:
def case6(MSFT):
    MSFT['Rolling 20'] = MSFT['Adj Close'].rolling(20).mean()
    MSFT['Adj Close'] = MSFT['Rolling 20']
    print(MSFT[['Adj Close']])

In [ ]:
MSFT['Rolling 20'] = MSFT['Adj Close'].rolling(20).mean()

In [ ]:
MSFT.tail()

### Case 7 - Volatility for a Period of 100

In [ ]:
def case7(MSFT):
    MSFT['daily_close'] = (MSFT['Adj Close']-MSFT['Adj Close'].shift(1))/MSFT['Adj Close'].shift(1)
    MSFT['Volatility_100'] = MSFT['daily_close'].rolling(100).std()*10
    MSFT['Adj Close'] = MSFT['Volatility_100']
    print(MSFT[['Adj Close']])

In [ ]:
# https://www.investopedia.com/ask/answers/021015/how-can-you-calculate-volatility-excel.asp

In [ ]:
MSFT['Volatility_100'] = MSFT['daily_close'].rolling(100).std()*10

### Case 8 - Signal Strategy

In [ ]:
def case8(MSFT):
    MSFT['short_mavg'] = MSFT.Close.rolling(window = 50, min_periods = 1).mean()
    MSFT['long_mavg'] = MSFT.Close.rolling(window = 100, min_periods = 1).mean()
    MSFT.loc[MSFT['short_mavg'] > MSFT['long_mavg'], 'signal'] = 1
    MSFT.loc[MSFT['short_mavg'] <= MSFT['long_mavg'], 'signal'] = 0
    MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)>0, 'orders'] = 1
    MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)<0, 'orders'] = -1
    MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)==0, 'orders'] = 0
    print(MSFT[['signal','short_mavg','long_mavg','orders']])

In [ ]:
MSFT.Close.rolling(window = 50, min_periods=1).mean()

In [ ]:
MSFT['short_mavg'] = MSFT.Close.rolling(window = 50, min_periods = 1).mean()
MSFT['long_mavg'] = MSFT.Close.rolling(window = 100, min_periods = 1).mean()

In [ ]:
MSFT.loc[MSFT['short_mavg'] > MSFT['long_mavg'], 'signal'] = 1
MSFT.loc[MSFT['short_mavg'] <= MSFT['long_mavg'], 'signal'] = 0

In [ ]:
MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)>0, 'order'] = 1
MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)<0, 'order'] = -1
MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)==0, 'order'] = 0

In [ ]:
MSFT.head()

### Case 9 - Protofolio

In [ ]:
# You will need to use the dataframe signals

    # You are going to set your initial amount of money you want
    # to invest --- here it is 10,000
    
    # You are going to create a new dataframe positions
    # Remember the index is still the same as signals
    
    # You are going to buy 10 shares of MSFT when signal is 1
    # You are going to sell 10 shares of MSFT when signal is -1
    # You will assign these values to the column MSFT of the
    # dataframe positions
    
    # You are now going to calculate the notional (quantity x price)
    # for your portfolio. You will multiply Adj Close from
    # the dataframe containing prices and the positions (10 shares)
    # You will store it into the variable portfolio
    
    # Add `holdings` to portfolio
    
    # You will store positions.diff into pos_diff
    
    # You will now add a column cash in your dataframe portfolio
    # which will calculate the amount of cash you have
    # initial_capital - (the notional you use for your different buy/sell)
    
    # You will now add a column total to your portfolio calculating the part of holding
    # and the part of cash
    # Add `returns` to portfolio
    # Print the first lines of `portfolio`

In [ ]:
def case9(MSFT):
    MSFT['short_mavg'] = MSFT.Close.rolling(window = 50, min_periods = 1).mean()
    MSFT['long_mavg'] = MSFT.Close.rolling(window = 100, min_periods = 1).mean()
    MSFT.loc[MSFT['short_mavg'] > MSFT['long_mavg'], 'signal'] = 1
    MSFT.loc[MSFT['short_mavg'] <= MSFT['long_mavg'], 'signal'] = 0
    MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)>0, 'orders'] = 1
    MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)<0, 'orders'] = -1
    MSFT.loc[MSFT['signal']-MSFT['signal'].shift(1)==0, 'orders'] = 0
    
    portfolio = MSFT[['Adj Close','signal','short_mavg','long_mavg','orders']].copy()
    portfolio['orders'] = portfolio['orders'].fillna(0)
    portfolio['quantity'] = portfolio['orders'].cumsum().fillna(0)*10
    portfolio['MSFT'] = portfolio['quantity']*portfolio['Adj Close']
    portfolio['holdings'] = portfolio['MSFT']
    portfolio['cash_change'] = -portfolio['orders']*portfolio['Adj Close']*10
    portfolio['cash'] = 10000 + portfolio['cash_change'].cumsum()
    portfolio['total'] = portfolio['holdings']+portfolio['cash']
    portfolio['returns'] = portfolio['total']/portfolio['total'].shift(1)-1
    print(portfolio[['MSFT','holdings','cash','total','returns']])

In [ ]:
portfolio = MSFT[['Adj Close','signal','short_mavg','long_mavg','order']].copy()

In [ ]:
portfolio.tail()

In [ ]:
portfolio['quantity'] = portfolio.loc[:,'order'].cumsum().fillna(0)*10
portfolio.head()

In [ ]:
portfolio['MSFT'] = portfolio['quantity']*portfolio['Adj Close']
portfolio.head()

In [ ]:
portfolio['holdings'] = portfolio['MSFT']
portfolio['cash_change'] = -portfolio['order']*portfolio['Adj Close']
portfolio['cash'] = 10000 + portfolio['cash_change'].cumsum()

In [ ]:
portfolio.tail()